Uploading a group of files with metdata stored in a CSV file
======================

We'll be using standard python tools to parse CSV and create XML documents describing files for BisQue

Make sure you have bisque api installed:
> pip install bisque-api

In [ ]:
import os
import csv
try:
    from lxml import etree
except ImportError:
    import xml.etree.ElementTree as etree

Include BisQue API

In [ ]:
from bqapi import BQSession
from bqapi.util import save_blob

Define some paths

In [ ]:
path = '.'
path_csv = os.path.join(path, 'BisQue_Seeds.csv')

Parse CSV file and define metadata fields
------------------------------------------

We'll create a list of dictionaries with file names and their metadata, this way each file could have its own metadata fields

In [ ]:
files = []
with open(path_csv, 'rb') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        f = {}
        for k,v in row.iteritems():
            f[k] = v
        print f
        files.append(f)

Create XML resources per image
------------------------------

Each file/image in the BisQue system is described with an XML document containing metadata, we'll create this document and post it to the system

In [ ]:
ignore = ['url', 'filename', 'resource']
for f in files:
    resource = etree.Element('image', name=f['filename'])
    for k,v in f.iteritems():
        if k not in ignore:
            t = etree.SubElement(resource, 'tag', name=k, value=v)
            if 'url' in k:
                t.set('type', 'link')
    print etree.tostring(resource, pretty_print=True)
    f['resource'] = resource

Upload images
--------------

Initialize a BisQue session using simple user credentials

In [ ]:
root = 'https://bisque.cyverse.org'
user = 'demo'
pswd = 'iplant'

session = BQSession().init_local(user, pswd,  bisque_root=root, create_mex=False)

In [ ]:
for f in files:
    print '\nuploading %s'%f['filename']
    filepath = os.path.join(path, f['filename'])
    resource = f['resource']

    # use import service to /import/transfer activating import service
    r = etree.XML(session.postblob(filepath, xml=resource)).find('./') 

    if r is None or r.get('uri') is None:
        print 'Upload failed'
    else:
        print 'Uploaded ID: %s, URL: %s'%(r.get('resource_uniq'), r.get('uri'))
        f['url'] = r.get('uri')

Create a dataset with all successfully uploaded files
-------------------------

In [ ]:
dataset = etree.Element('dataset', name='demo seeds')
for f in files:
    if 'url' not in f:
        continue
    v = etree.SubElement(dataset, 'value', type="object")
    v.text = f['url']
print etree.tostring(dataset, pretty_print=True)

Upload dataset
---------------

In [ ]:
url = session.service_url('data_service')
r = session.postxml(url, dataset)

if r is None or r.get('uri') is None:
    print 'Dataset failed'
else:
    print 'Dataset ID: %s, URL: %s'%(r.get('resource_uniq'), r.get('uri'))
    f['url'] = r.get('uri')